# Extracción de datos de precipitación desde repositorio Cassandra

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 193 de 2023. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

In [1]:
import numpy as np
import pandas as pd
import datetime
import prestodb 
import matplotlib.pyplot as plt
import os
from datetime import datetime

In [2]:
conn=prestodb.dbapi.connect(
    host='172.16.50.20',
    port=8080,
    user='Paola',
    catalog='raw',
    schema='cassandra',
)
cur = conn.cursor()

_________________

El siguiente código permite hacer la descarga diaria de datos de precipitación

In [4]:
def descarga_datoscrud_cassandra(inicio, fin, codestacion, sensor):
    cur.execute(f''' select lm.station, s.name, lm.sensor, lm.event_time + interval '5' hour event_time, lm.event_value 
    from cassandra.raw.weather_events AS lm INNER JOIN cassandra.raw.stations AS s
        ON lm.station = s.stationid
    where lm.station in ('{codestacion}')
    AND lm.event_time BETWEEN timestamp '{inicio}' AND timestamp '{fin}' 
    AND lm.sensor in ('{sensor}') ''') # estación de la forma '0021206600'; sensor de la forma '0240'
    station_sensem = cur.fetchall()
    
    # Se genera un Data Frame a partir de los datos extaídos
    stationdf = pd.DataFrame(station_sensem, columns= ['Station', 'Name', 'Sensor','Fecha','Valor'])
    print(stationdf.head(3))
    
    # Se genera la lista de las diferentes estaciones con base en sus códigos
    j = stationdf['Station'].values.ravel()
    k = pd.unique(j).tolist()

    # Se crea una carpeta con el nombre del semestre
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d') #'%Y-%m-%d %H:%M:%S.%f'
    fin_date = datetime.strptime(fin, '%Y-%m-%d') #'%Y-%m-%d %H:%M:%S.%f'
    semestre = inicio_date.strftime('%Y-%m-%d') + "_a_" + fin_date.strftime('%Y-%m-%d')
    if not os.path.exists(semestre):
        os.makedirs(semestre)

    # Se exportan los datos separados por estaciones según su código para futuros usos 
    for i in k:
        Filtro = stationdf['Station'] == i
        h = stationdf[Filtro]
        nombre_archivo = semestre + "/Estacion_" + str(i) + ".csv"
        print('Se guardó el archivo:', nombre_archivo)
        h.to_csv(nombre_archivo)
    
    return station_sensem, stationdf

In [10]:
# Se llama la función con parámetros especificados con los siguientes formatos
EMA_0240_13_01, df_EMA_0240_23_08 = descarga_datoscrud_cassandra('2018-09-01', '2018-09-30', '0023100070', 'PTPM_CON')

      Station     Name    Sensor                    Fecha  Valor
0  0023100070  RAMBLAS  PTPM_CON  2018-09-01 07:00:00.000    0.0
1  0023100070  RAMBLAS  PTPM_CON  2018-09-02 07:00:00.000   80.0
2  0023100070  RAMBLAS  PTPM_CON  2018-09-03 07:00:00.000    0.0
Se guardó el archivo: 2018-09-01_a_2018-09-30/Estacion_0023100070.csv


In [3]:
# Sin estación específica
def desc_datoscrud_cassandra_noest(inicio, fin, sensor):
    cur.execute(f''' select lm.station, s.name, lm.sensor, lm.event_time + interval '5' hour event_time, lm.event_value 
    from cassandra.raw.weather_events AS lm INNER JOIN cassandra.raw.stations AS s
        ON lm.station = s.stationid
    where lm.event_time BETWEEN timestamp '{inicio}' AND timestamp '{fin}' 
    AND lm.sensor in ('{sensor}') ''') # estación de la forma '0021206600'; sensor de la forma '0240'
    station_sensem = cur.fetchall()
    
    # Se genera un Data Frame a partir de los datos extaídos
    stationdf = pd.DataFrame(station_sensem, columns= ['Station', 'Name', 'Sensor','Fecha','Valor'])
    print(stationdf.head(3))
    
    # Se genera la lista de las diferentes estaciones con base en sus códigos
    j = stationdf['Station'].values.ravel()
    k = pd.unique(j).tolist()

    # Se crea una carpeta con el nombre del semestre
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d') #'%Y-%m-%d %H:%M:%S.%f'
    fin_date = datetime.strptime(fin, '%Y-%m-%d') #'%Y-%m-%d %H:%M:%S.%f'
    semestre = inicio_date.strftime('%Y-%m-%d') + "_a_" + fin_date.strftime('%Y-%m-%d')
    if not os.path.exists(semestre):
        os.makedirs(semestre)

    # Se exportan los datos separados por estaciones según su código para futuros usos 
    for i in k:
        Filtro = stationdf['Station'] == i
        h = stationdf[Filtro]
        nombre_archivo = semestre + "/Estacion_" + str(i) + ".csv"
        print('Se guardó el archivo:', nombre_archivo)
        h.to_csv(nombre_archivo)
    
    return station_sensem, stationdf

In [6]:
# Se llama la función con parámetros especificados con los siguientes formatos
EMA_0240_13_01, df_EMA_0240_23_08 = desc_datoscrud_cassandra_noest('2018-01-01','2018-01-02', 'VVAG_CON')

      Station                     Name    Sensor                    Fecha  \
0  0014015080  AEROPUERTO RAFAEL NUNEZ  VVAG_CON  2018-01-01 05:00:00.000   
1  0014015080  AEROPUERTO RAFAEL NUNEZ  VVAG_CON  2018-01-01 06:00:00.000   
2  0014015080  AEROPUERTO RAFAEL NUNEZ  VVAG_CON  2018-01-01 07:00:00.000   

   Valor  
0    1.4  
1    1.4  
2    1.7  
Se guardó el archivo: 2018-01-01_a_2018-01-02/Estacion_0014015080.csv
Se guardó el archivo: 2018-01-01_a_2018-01-02/Estacion_0021205420.csv
Se guardó el archivo: 2018-01-01_a_2018-01-02/Estacion_0034015010.csv


In [ ]:
# Se llama la función con parámetros especificados con los siguientes formatos
EMA_0240_13_01, df_EMA_0240_23_08 = desc_datoscrud_cassandra_noest('2018-01-01','2018-01-02', '0104')

-----

In [44]:
df = pd.read_csv('2019-01-01_a_2019-01-04/Estacion_0035075040.csv')
df

,Unnamed: 0,Station,Name,Sensor,Fecha,Valor
0,0,35075040,INSTITUCION AGRICOLA MACANAL,TSSM_CON,2019-01-01 07:00:00.000,13.6
1,1,35075040,INSTITUCION AGRICOLA MACANAL,TSSM_CON,2019-01-01 13:00:00.000,23.2
2,2,35075040,INSTITUCION AGRICOLA MACANAL,TSSM_CON,2019-01-01 18:00:00.000,17.8
3,3,35075040,INSTITUCION AGRICOLA MACANAL,TSSM_CON,2019-01-02 07:00:00.000,14.4


In [45]:
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')

In [46]:
df['Mes'] = df['Fecha'].dt.month
df

,Unnamed: 0,Station,Name,Sensor,Fecha,Valor,Mes
0,0,35075040,INSTITUCION AGRICOLA MACANAL,TSSM_CON,2019-01-01 07:00:00,13.6,1
1,1,35075040,INSTITUCION AGRICOLA MACANAL,TSSM_CON,2019-01-01 13:00:00,23.2,1
2,2,35075040,INSTITUCION AGRICOLA MACANAL,TSSM_CON,2019-01-01 18:00:00,17.8,1
3,3,35075040,INSTITUCION AGRICOLA MACANAL,TSSM_CON,2019-01-02 07:00:00,14.4,1


In [17]:
df['Mes'].value_counts()

Mes
1    4
Name: count, dtype: int64

In [47]:
normales = pd.read_excel('NormClimatolEstándar_PRECIPITACION_9120.xlsx')

In [48]:
#normales['Station'] = normales['Station'].astype('int64')
normales['Station'].dtypes

dtype('int64')

In [39]:
normales['Station']

0       47067010
1       47060010
2       44187030
3       44187020
4       44157040
          ...   
1439    38015030
1440    38017040
1441    35260010
1442    35260030
1443    35267030
Name: Station, Length: 1444, dtype: int64

In [40]:
df['Station'].dtypes

dtype('int64')

In [49]:
# Crear una función para extraer el valor normal del mes correspondiente
def get_normal_value(row, normales_df):
    # Usar el mes para seleccionar la columna correcta
    month_column = row['Mes']
    # Encontrar el valor normal para la estación y el mes específicos
    normal_value = normales_df.loc[normales_df['Station'] == row['Station'], month_column]
    print(normal_value)
    return normal_value.values[0] if not normal_value.empty else None

# Aplicar la función para crear una nueva columna con el valor normal
df['Valor_Normal'] = df.apply(get_normal_value, normales_df=normales, axis=1)
# Calcular la nueva columna como solicitaste
df['Índice'] = (df['Valor'] * 100) / df['Valor_Normal']
print(df)

297    42.003704
Name: 1, dtype: float64
297    42.003704
Name: 1, dtype: float64
297    42.003704
Name: 1, dtype: float64
297    42.003704
Name: 1, dtype: float64
   Unnamed: 0   Station                          Name    Sensor  \
0           0  35075040  INSTITUCION AGRICOLA MACANAL  TSSM_CON   
1           1  35075040  INSTITUCION AGRICOLA MACANAL  TSSM_CON   
2           2  35075040  INSTITUCION AGRICOLA MACANAL  TSSM_CON   
3           3  35075040  INSTITUCION AGRICOLA MACANAL  TSSM_CON   

                Fecha  Valor  Mes  Valor_Normal     Índice  
0 2019-01-01 07:00:00   13.6    1     42.003704  32.378097  
1 2019-01-01 13:00:00   23.2    1     42.003704  55.233225  
2 2019-01-01 18:00:00   17.8    1     42.003704  42.377215  
3 2019-01-02 07:00:00   14.4    1     42.003704  34.282691  


In [43]:
var = '1234'
var.astype('int64')

AttributeError: 'str' object has no attribute 'astype'